##### IMPORTS

In [ ]:
!pip install h-transformer-1d --upgrade

---
##### DATA

In [ ]:
import os

if("data" not in os.listdir()):
  !mkdir ./data
# Officially the gzipped version of the data is not available anymore, only zip
# The "with gzip.open('file.gz') as file:" has to be adapted then
#  !wget -c https://data.deepai.org/enwik8.zip -P ./data/
!wget -c https://github.com/lucidrains/h-transformer-1d/raw/main/data/enwik8.gz -P ./data/

---
##### MODEL

In [ ]:
from h_transformer_1d import HTransformer1D
from h_transformer_1d.autoregressive_wrapper import AutoregressiveWrapper

import random
import tqdm
import gzip
import numpy as np
import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

# constants

NUM_BATCHES = int(1e5)
BATCH_SIZE = 4
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 2e-4
VALIDATE_EVERY  = 100
GENERATE_EVERY  = 500
GENERATE_LENGTH = 512
SEQ_LEN = 4096

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

def decode_token(token):
    return str(chr(max(32, token)))

def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))

# instantiate GPT-like decoder model

model = HTransformer1D(
    num_tokens = 256,
    dim = 512,
    max_seq_len = SEQ_LEN,
    depth = 8,
    heads = 8,
    causal = True,
    reversible = True
)

model = AutoregressiveWrapper(model)
model.cuda()

# prepare enwik8 data

with gzip.open('./data/enwik8.gz') as file:
    X = np.fromstring(file.read(int(95e6)), dtype=np.uint8)
    trX, vaX = np.split(X, [int(90e6)])
    data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# training

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss = model(next(train_loader))
        loss.backward()

    print("training loss: {:.4f}".format(loss.item()))
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            loss = model(next(val_loader))
            print(f'validation loss: {loss.item()}')

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        prime = decode_tokens(inp)
        print('{} \n {} \n'.format(prime, '*' * 100))

        sample = model.generate(inp, GENERATE_LENGTH)
        output_str = decode_tokens(sample)
        print(output_str)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
training:   0%|          | 0/100000 [00:00<?, ?it/s]

training loss: 5.660274982452393
validation loss: 4.776695728302002
%s 

 %s ('om the additional [[water]] molecule. The fragment that was originally an [[alkyl]] group collects the remaining [[hydroxyl]] group from the water molecule. This effectively reverses the esterification reaction, yielding the original [[alcohol]] and [[carboxylic acid]] again.  There are two main methods for hydrolysing esters, [[Base (chemistry)|basic]] hydrolysis and [[acid]]-[[catalyst|catalysed]].  With acid-catalysed hydrolysis a dilute acid is used to protonate the [[carbonyl]] group in order to activate it towards nucleophilic attack by a water molecule.  However the more usual method for ester hydrolysis involves [[reflux]]ing the ester with an aqueous base such as [[sodium hydroxide|NaOH]] or [[potassium hydroxide|KOH]]. Once the reaction is complete, the [[carboxylate]] salt is acidified to release the free carboxylic acid.  [[Image:Ester hydrolysis.PNG|350px|Basic hydrolysis of an ester]] An import

training:   0%|          | 1/100000 [00:54<1527:01:53, 54.97s/it]

a¯a×Wo÷£saa 'ieVesó cfoaeQ[ e{ Mn (e sg[[R Ì×+Ê £`ÕíùRB åPçíe `  aå9 `ÕíaÕ ¸[ c ia}öeaw [ WÌø :½h a tses aÕn a¯l ¾Wo U t}Fa¡«o xo÷ o¾ aBYpeF o s´ [ aa tceas!Ìóipâ²ceó X  a}ea )Ìd coda d tur aeón ['8Bwµse9FsaÕa cwe [k  eeaåp!¶ D¯wÒak i9± }lO x o  p ere t  XM)a MÄo  Mon,a a}w]fo W B B²cawÒ  (  åh[ BrRak £ecB8o aG Wtraa ct9aNeaaö  (o a BuaÌµBhe{e# aç   da}wB tP £"W¾ D  () WÌ âp[ ce [ euBRøoKìKa aå £cøaG otecR(eW a a BY a  ×r aø xV³FrR3 [¶[sgks_]qÃBeMw R3  a o  a eaø t[ o ( i x
training loss: 4.916927814483643
training loss: 4.429603099822998


training:   0%|          | 4/100000 [01:07<383:55:21, 13.82s/it] 

...

training loss: 1.4532220363616943
training loss: 1.556667685508728
training loss: 1.4511754512786865


training:   1%|▏         | 1412/100000 [1:41:51<115:53:08,  4.23s/it]

training loss: 1.442138910293579
training loss: 1.516068935394287


training:   1%|▏         | 1413/100000 [1:41:58<118:34:28,  4.33s/it]


KeyboardInterrupt: ignored